In [3]:
# Step 1: Install necessary packages
!pip install streamlit pyngrok opencv-python-headless ultralytics transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [9]:
!pip install streamlit -q


In [10]:
!pip install pyngrok


In [25]:


# Step 2: Create a Python script for the Streamlit app using %writefile
%%writefile app_1.py
import streamlit as st
import cv2
import os
import json
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Function to process video and extract class information
def process_video(video_path, output_folder):
    # Load the YOLOv8 model
    model = YOLO("yolov8n.pt")

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not cap.isOpened():
        st.error("Error: Could not open video file.")
        return None

    # Get the video properties
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_seconds = total_frames / fps
    interval = 1  # Interval in seconds

    frame_index = 0
    max_frames = int(duration_seconds)  # Number of frames to process

    # Create a dictionary to store the detected information
    detected_info_dict = {}

    # Loop through the video frames at fixed intervals
    while cap.isOpened() and frame_index < max_frames:
        # Set the position of the next frame to read
        frame_position = int(fps * frame_index * interval)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_position)

        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True)

            # Extract detection results
            detections = results[0].boxes
            detected_info = []

            for detection in detections:
                class_id = int(detection.cls)
                class_name = results[0].names[class_id]  # Use class names from YOLO output

                # Ensure proper extraction of confidence and coordinates
                confidence = detection.conf.item()  # Convert to Python float
                coordinates = detection.xyxy.cpu().numpy().tolist()  # Convert to list

                detected_info.append({
                    "class": class_name,
                    "confidence": confidence,
                    "coordinates": coordinates
                })

            # Store the detected information in the dictionary
            detected_info_dict[f"frame_{frame_index}"] = detected_info

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Display the annotated frame in Google Colab
            cv2_imshow(annotated_frame)

            frame_index += 1
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object
    cap.release()

    # Save the detected information dictionary to a file
    with open(os.path.join(output_folder, "detected_info.json"), "w") as file:
        json.dump(detected_info_dict, file, indent=2)

    return detected_info_dict

# Function to extract classes from detected information
def extract_classes(detected_info_dict):
    class_dict = {}
    frame_count = 0  # Initialize frame count

    for frame_number, detected_objects in detected_info_dict.items():
        frame_count += 1  # Increment frame count
        # Extract class names for the current frame
        class_names = [obj['class'] for obj in detected_objects]
        class_dict[frame_number] = class_names  # Store in the dictionary

    return class_dict, frame_count

# Streamlit app
st.title("YOLOv8 Video Analysis")

# Upload video file
uploaded_file = st.file_uploader("Upload a video file", type=["mp4", "avi", "mov"])

if uploaded_file is not None:
    # Create temporary directories for input and output
    input_folder = "temp_input"
    output_folder = "temp_output"
    os.makedirs(input_folder, exist_ok=True)
    os.makedirs(output_folder, exist_ok=True)

    # Save uploaded video to temporary file
    video_path = os.path.join(input_folder, uploaded_file.name)
    with open(video_path, "wb") as f:
        f.write(uploaded_file.read())

    # Display the video using Streamlit's video player
    st.video(video_path)

    # Process video when submit button is clicked
    if st.button("Submit"):
        st.write("Processing video...")
        detected_info_dict = process_video(video_path, output_folder)

        if detected_info_dict is not None:
            class_values, frame_count = extract_classes(detected_info_dict)
            st.write(f"Total number of frames: {frame_count}")
            st.write("List of detected classes:")
            st.json(class_values)
        else:
            st.error("Error processing the video.")


Overwriting app_1.py


In [29]:
from pyngrok import ngrok

# Set authentication token if you haven't already done so
ngrok.set_auth_token("2jooVXb8DvqbenWY0TgIeBBjpi4_39qryxcypQgy6BcgxpPg6")

# Start Streamlit server on a specific port
!nohup streamlit run app_1.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The authtoken credential '2jollpJ7jtoAH2IgF1Df20DWFDT' has been revoked\nand is no longer valid.\r\n\r\nERR_NGROK_300\r\n"}}
